# Notebook de Selección de Características (Feature Selection)

Este notebook implementa una variedad de técnicas de selección de características solicitadas, aplicadas al fichero `cleaned_main_financial_metrics.csv`.

**Aviso Importante:** La lista proporcionada es extremadamente extensa y académica.
* **Implementados:** Se implementarán los métodos más comunes y accesibles que se encuentran en las librerías estándar de Python (`scikit-learn`, `scipy`).
* **No Implementados:** Muchos métodos (ej. Bi-normal separation, TNoM, GRASP, VNS, Scatter Search, ACO, PSO, Algoritmos de Estimación de Distribución, etc.) son muy especializados, no están en `scikit-learn` y requerirían implementaciones personalizadas complejas o librerías de nicho. Se dejará constancia de ellos.


## 0. Importar Librerías

In [12]:
import pandas as pd
import numpy as np
import time
import warnings
from matplotlib import pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')


# Filter Methods
from sklearn.feature_selection import SelectKBest, f_classif, chi2, mutual_info_classif
from scipy.stats import mannwhitneyu, kruskal

# Wrapper Methods
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.metrics import accuracy_score, make_scorer

# Models (Non-probabilistic)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

## 1. Carga y Preparación de Datos

In [13]:
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler, OneHotEncoder


TARGET_VARIABLE = 'recommendationClass'
COLUMNS_TO_DROP = [
    'recommendationKey', 'recommendationMean',
    'shortName', 'symbol', 'Ticker', 'fullExchangeName', 'twoHundredDayAverage',
]

df = pd.read_csv('../cleaned_main_financial_metrics.csv')

ALL_COLS = df.columns.tolist()
FEATURE_COLS = [col for col in ALL_COLS if col not in COLUMNS_TO_DROP and col != TARGET_VARIABLE]

NUMERIC_FEATURES = df[FEATURE_COLS].select_dtypes(include=['float64']).columns.tolist()
CATEGORICAL_FEATURES = df[FEATURE_COLS].select_dtypes(include=['object', 'bool', 'int32', 'int64']).columns.tolist()

existing_cols_to_drop = [col for col in COLUMNS_TO_DROP if col in df.columns]
df_processed = df.drop(columns=existing_cols_to_drop)

y = df_processed[TARGET_VARIABLE]
X = df_processed.drop(columns=[TARGET_VARIABLE])

le = LabelEncoder()
y_encoded = le.fit_transform(y)
print(f"Clases del objetivo: {list(le.classes_)}")

numeric_features = X[NUMERIC_FEATURES].columns.tolist()
categorical_features = X[CATEGORICAL_FEATURES].columns.tolist()

print(f"Características numéricas ({len(numeric_features)}): {numeric_features}")
print(f"Características categóricas ({len(categorical_features)}): {categorical_features}")

for col in categorical_features:
    X[col] = LabelEncoder().fit_transform(X[col])    

Clases del objetivo: [np.int64(0), np.int64(1), np.int64(2), np.int64(3), np.int64(4)]
Características numéricas (47): ['numberOfAnalystOpinions', 'currentPrice', 'allTimeHigh', 'allTimeLow', '52WeekChange', 'fiftyDayAverageChangePercent', 'twoHundredDayAverageChangePercent', 'beta', 'averageVolume', 'marketCap', 'enterpriseValue', 'priceToBook', 'enterpriseToRevenue', 'profitMargins', 'grossMargins', 'ebitdaMargins', 'operatingMargins', 'returnOnAssets', 'returnOnEquity', 'revenueGrowth', 'totalRevenue', 'revenuePerShare', 'grossProfits', 'ebitda', 'netIncomeToCommon', 'trailingEps', 'totalCash', 'totalCashPerShare', 'totalDebt', 'quickRatio', 'currentRatio', 'bookValue', 'operatingCashflow', 'freeCashflow', 'trailingAnnualDividendYield', 'payoutRatio', 'sharesOutstanding', 'floatShares', 'sharesShort', 'sharesPercentSharesOut', 'shortRatio', 'shortPercentOfFloat', 'heldPercentInsiders', 'heldPercentInstitutions', 'fullTimeEmployees', '_debtToEquity', '_PER']
Características categóric

In [14]:
# --- Split train/test estratificado ---
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.3, random_state=42, stratify=y_encoded)

In [15]:
# # --- Seleccionar columnas categóricas para One-Hot ---
# encoder = OneHotEncoder(sparse_output=False, drop='first', handle_unknown='ignore')

# cat_cols = [
#     col for col in df[FEATURE_COLS].select_dtypes(include=['object', 'int32', 'int64']).columns
#     if col in X_train.columns and (
#         X_train[col].nunique() > 2 or
#         (X_train[col].nunique() == 2 and set(X_train[col].dropna().unique()) != {0, 1})
#     )
# ]

# binary_cat_cols = [col for col in X_train.columns if col not in cat_cols and X_train[col].nunique() == 2]

# # --- One-Hot encode -----------------------------------------------------
# X_train_encoded_cat = pd.DataFrame(
#     encoder.fit_transform(X_train[cat_cols]),
#     columns=encoder.get_feature_names_out(cat_cols),
#     index=X_train.index
# )

# X_test_encoded_cat = pd.DataFrame(
#     encoder.transform(X_test[cat_cols]),
#     columns=encoder.get_feature_names_out(cat_cols),
#     index=X_test.index
# )

# processed_cat_columns = X_train_encoded_cat.columns.tolist() + binary_cat_cols

# X_train_categorical = pd.concat([X_train[binary_cat_cols], X_train_encoded_cat], axis=1)
# X_test_categorical = pd.concat([X_test[binary_cat_cols], X_test_encoded_cat], axis=1)

processed_cat_columns = CATEGORICAL_FEATURES.copy()

X_train_categorical = X_train[processed_cat_columns].copy()
X_test_categorical = X_test[processed_cat_columns].copy()



In [16]:
# --- STANDARD SCALER -----------------------------------------------------
scaler_std = StandardScaler()
X_train_scaled_num = pd.DataFrame(
    scaler_std.fit_transform(X_train[numeric_features]),
    columns=numeric_features,
    index=X_train.index
)
X_test_scaled_num = pd.DataFrame(
    scaler_std.transform(X_test[numeric_features]),
    columns=numeric_features,
    index=X_test.index
)

# Combinar numéricas, binarias y One-Hot
X_train_std_final = pd.concat([X_train_scaled_num, X_train_categorical], axis=1)
X_test_std_final = pd.concat([X_test_scaled_num, X_test_categorical], axis=1)

# Añadir target
train_std_final = X_train_std_final.copy()
test_std_final = X_test_std_final.copy()
train_std_final["target"] = y_train
test_std_final["target"] = y_test

# Guardar CSV StandardScaler
train_std_final.to_csv("train_std_main_financial_metrics.csv", index=False)
test_std_final.to_csv("test_std_main_financial_metrics.csv", index=False)

print("✅ Archivos guardados con StandardScaler")
print("train_std_main_financial_metrics.csv")
print("test_std_main_financial_metrics.csv")

# --- MIN-MAX SCALER ------------------------------------------------------
scaler_minmax = MinMaxScaler()
X_train_minmax_num = pd.DataFrame(
    scaler_minmax.fit_transform(X_train[numeric_features]),
    columns=numeric_features,
    index=X_train.index
)

X_test_minmax_num = pd.DataFrame(
    scaler_minmax.transform(X_test[numeric_features]),
    columns=numeric_features,
    index=X_test.index
)

# Combinar numéricas, binarias y One-Hot
X_train_minmax_final = pd.concat([X_train_minmax_num, X_train_categorical], axis=1)
X_test_minmax_final = pd.concat([X_test_minmax_num, X_test_categorical], axis=1)

# Añadir target
train_minmax_final = X_train_minmax_final.copy()
test_minmax_final = X_test_minmax_final.copy()
train_minmax_final["target"] = y_train
test_minmax_final["target"] = y_test

# Guardar CSV MinMaxScaler
train_minmax_final.to_csv("train_minmax_main_financial_metrics.csv", index=False)
test_minmax_final.to_csv("test_minmax_main_financial_metrics.csv", index=False)

print("✅ Archivos guardados con MinMaxScaler")
print("train_minmax_main_financial_metrics.csv")
print("test_minmax_main_financial_metrics.csv")

✅ Archivos guardados con StandardScaler
train_std_main_financial_metrics.csv
test_std_main_financial_metrics.csv
✅ Archivos guardados con MinMaxScaler
train_minmax_main_financial_metrics.csv
test_minmax_main_financial_metrics.csv


In [17]:
from imblearn.over_sampling import SMOTE

# --- SMOTE sobre train (solo StandardScaler como ejemplo) ---
smote = SMOTE(random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(
    train_std_final.drop(columns=["target"]),
    train_std_final["target"]
)

train_balanced_df = X_train_balanced.copy()
train_balanced_df["target"] = y_train_balanced

# Guardar CSV reequilibrado
display(train_balanced_df["target"].value_counts())
train_balanced_df.to_csv("train_std_balanced.csv", index=False)
print("✅ Train reequilibrado con SMOTE guardado: train_std_balanced.csv")

target
2    317
0    317
3    317
4    317
1    317
Name: count, dtype: int64

✅ Train reequilibrado con SMOTE guardado: train_std_balanced.csv


In [18]:
# Definir conjuntos finales para modelado. Esta vez usaré std scaler
X_train = X_train_std_final.copy()
X_test = X_test_std_final.copy()
y_train = train_std_final['target'].copy()
y_test = test_std_final['target'].copy()


# Eliminar variables del namespace
del X_train_std_final, X_test_std_final
del X_train_minmax_final, X_test_minmax_final
del train_std_final, test_std_final
del train_minmax_final, test_minmax_final

# Forzar recolección de basura
import gc
gc.collect()


0

# 2. Métodos de Filtro (Filter Methods)

## 2.1 Métodos Paramétricos

### Predictores Continuos: ANOVA (t-test)
`f_classif` de Scikit-learn implementa un test F de ANOVA. Si el problema es de clasificación binaria, esto es equivalente a un **t-test**.


In [19]:
print("--- 2.1.1 Filter: ANOVA F-test (f_classif) ---")
selector_anova = SelectKBest(score_func=f_classif, k=K_BEST)
selector_anova.fit(X_train_scaled, y_train)

# Obtener resultados
selected_indices_anova = selector_anova.get_support(indices=True)
selected_features_anova = X.columns[selected_indices_anova]
scores_anova = selector_anova.scores_[selected_indices_anova]

results_anova = pd.DataFrame({'Feature': selected_features_anova, 'F-Score': scores_anova}).sort_values(by='F-Score', ascending=False)
print(f"Mejores {K_BEST} características según ANOVA (f_classif):")
print(results_anova)


--- 2.1.1 Filter: ANOVA F-test (f_classif) ---


NameError: name 'K_BEST' is not defined

### Predictores Discretos: Chi-cuadrado (Chi-squared) e Información Mutua

* **Chi-cuadrado (`chi2`)**: Requiere características no negativas. Usaremos los datos escalados con `MinMaxScaler`.
* **Información Mutua (`mutual_info_classif`)**: Es un método potente, técnicamente no paramétrico, pero `sklearn` lo agrupa aquí. No requiere escalado específico, pero funciona mejor con datos discretos o escalados (como MinMax).


In [ ]:
print(f"\n--- 2.1.2 Filter: Chi-cuadrado (chi2) ---")
# Usamos X_train_minmax (no negativos)
selector_chi2 = SelectKBest(score_func=chi2, k=K_BEST)
selector_chi2.fit(X_train_minmax, y_train)

# Obtener resultados
selected_indices_chi2 = selector_chi2.get_support(indices=True)
selected_features_chi2 = X.columns[selected_indices_chi2]
scores_chi2 = selector_chi2.scores_[selected_indices_chi2]

results_chi2 = pd.DataFrame({'Feature': selected_features_chi2, 'Chi2-Score': scores_chi2}).sort_values(by='Chi2-Score', ascending=False)
print(f"Mejores {K_BEST} características según Chi-cuadrado:")
print(results_chi2)

print(f"\n--- 2.1.3 Filter: Información Mutua (mutual_info_classif) ---")
# Usamos X_train_minmax (buena práctica para MI)
selector_mi = SelectKBest(score_func=mutual_info_classif, k=K_BEST)
selector_mi.fit(X_train_minmax, y_train)

# Obtener resultados
selected_indices_mi = selector_mi.get_support(indices=True)
selected_features_mi = X.columns[selected_indices_mi]
scores_mi = selector_mi.scores_[selected_indices_mi]

results_mi = pd.DataFrame({'Feature': selected_features_mi, 'MI-Score': scores_mi}).sort_values(by='MI-Score', ascending=False)
print(f"Mejores {K_BEST} características según Información Mutua:")
print(results_mi)



--- 2.1.2 Filter: Chi-cuadrado (chi2) ---
Mejores 10 características según Chi-cuadrado:
                       Feature  Chi2-Score
1                     In_SP500   37.134766
9                 has_benefits   17.621138
2                    In_NASDAQ   16.986076
5  trailingAnnualDividendYield   14.622993
7          shortPercentOfFloat    9.050226
3          enterpriseToRevenue    7.740894
6       sharesPercentSharesOut    6.459521
0      numberOfAnalystOpinions    6.148590
8          heldPercentInsiders    5.762688
4                    totalDebt    3.063679

--- 2.1.3 Filter: Información Mutua (mutual_info_classif) ---
Mejores 10 características según Información Mutua:
                       Feature  MI-Score
1                 52WeekChange  0.079091
9          shortPercentOfFloat  0.064126
3                revenueGrowth  0.062290
7            operatingCashflow  0.060976
0                   allTimeLow  0.055693
4            netIncomeToCommon  0.055190
2               returnOnAssets  0.0

### Métodos Paramétricos (No implementados en `sklearn`)
* **Gain ratio, Symmetrical uncertainty, Odds ratio, Bi-normal separation**: Estos métodos no están disponibles en `scikit-learn` y requerirían implementaciones manuales o librerías de terceros (ej. `skfeature`).


## 2.2 Métodos No Paramétricos (Model-Free)

### Mann-Whitney y Kruskal-Wallis
Son las alternativas no paramétricas al t-test y ANOVA, respectivamente. Se usan cuando no se asume una distribución normal. Los aplicaremos desde `scipy.stats`.

* **Mann-Whitney U test**: Para 2 clases.
* **Kruskal-Wallis H test**: Para 2 o más clases.


In [ ]:
print("\n--- 2.2.1 Filter: Kruskal-Wallis / Mann-Whitney ---")
# Usamos los datos originales (no escalados), ya que estos tests se basan en rangos.
n_classes = len(np.unique(y_train))
p_values = []
feature_names = X_train.columns

if n_classes == 2:
    print("Aplicando Mann-Whitney U test (2 clases)")
    group1 = X_train[y_train == 0]
    group2 = X_train[y_train == 1]
    for col in feature_names:
        # Añadimos 'alternative='two-sided'' para compatibilidad con scipy moderno
        try:
            stat, p = mannwhitneyu(group1[col], group2[col], alternative='two-sided')
            p_values.append(p)
        except ValueError:
            p_values.append(1.0) # Asignar p-value alto si hay error (ej. varianza cero)
else:
    print(f"Aplicando Kruskal-Wallis H test ({n_classes} clases)")
    groups = [X_train[y_train == i] for i in np.unique(y_train)]
    for i, col in enumerate(feature_names):
        samples = [g.iloc[:, i] for g in groups]
        try:
            stat, p = kruskal(*samples)
            p_values.append(p)
        except ValueError:
            p_values.append(1.0)

# Un p-value más bajo significa que la característica es más discriminativa
results_nonparam = pd.DataFrame({'Feature': feature_names, 'p-value': p_values}).sort_values(by='p-value', ascending=True)
print(f"Mejores {K_BEST} características según el test No Paramétrico (p-value más bajo):")
print(results_nonparam.head(K_BEST))



--- 2.2.1 Filter: Kruskal-Wallis / Mann-Whitney ---
Aplicando Kruskal-Wallis H test (5 clases)
Mejores 10 características según el test No Paramétrico (p-value más bajo):
                        Feature       p-value
19                revenueGrowth  3.354177e-20
22                 grossProfits  1.386005e-17
32            operatingCashflow  3.716922e-16
20                 totalRevenue  8.791060e-16
23                       ebitda  3.767622e-15
28                    totalDebt  2.398883e-14
34  trailingAnnualDividendYield  5.887412e-13
10              enterpriseValue  6.776325e-13
4                  52WeekChange  1.802058e-12
41          shortPercentOfFloat  1.981951e-12


### Métodos No Paramétricos (No implementados en `sklearn`)
* **Threshold number of misclassification (TNoM)**
* **P-metric**
* **Between-groups to within-groups sum of squares** (Relacionado con ANOVA, pero implementaciones específicas pueden variar)
* **Scores based on estimating density functions**

Estos son métodos especializados que requieren implementación manual.


# 4. Métodos Wrapper (Wrapper Methods)

## 4.0 Definición de Modelos No Probabilísticos

Estos son los modelos que se utilizarán como evaluadores en los métodos Wrapper.
* k-nearest neighbors (KNN)
* Classification trees (Decision Tree)
* Artificial neural networks (ANN / MLP)
* Support vector machines (SVM)
* Rule induction (No existe en sklearn, usaremos `DecisionTreeClassifier` como la aproximación más cercana).


In [ ]:
# K-Features a seleccionar (para los ejemplos)
K_BEST = 10 

# Diccionario de modelos
# Usamos configuraciones más simples para que los wrappers se ejecuten más rápido
models = {
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5),
    "SVM (Linear)": SVC(kernel='linear', random_state=42, probability=False), # Kernel lineal es más rápido
    "ANN (MLP)": MLPClassifier(hidden_layer_sizes=(50,), max_iter=300, random_state=42) # Red pequeña
}

print("Modelos base definidos.")


Modelos base definidos.


## 4.1 Heurísticas Determinísticas

### Métodos Secuenciales (SFS y SBE)
`SequentialFeatureSelector` de Scikit-learn (SFS) puede operar en ambas direcciones:
* **Sequential Forward Selection (`direction='forward'`)**: Empieza sin características y añade la mejor en cada paso.
* **Sequential Backward Elimination (`direction='backward'`)**: Empieza con todas las características y elimina la peor en cada paso.

**Nota:** Estos métodos son *muy* lentos. Usaremos un modelo rápido (`DecisionTree`) y un número bajo de características a seleccionar (`K_WRAPPER = 5`) para este ejemplo.


In [ ]:
print("\n--- 4.1.1 Wrapper: Sequential Methods ---")
K_WRAPPER = 5 # Usar un K pequeño, SBE es muy costoso
model_wrapper = models["DecisionTree"]
# Usamos datos escalados (X_train_scaled) ya que los modelos (excepto DT) lo prefieren.
# Aunque DT es invariante, es buena práctica mantener consistencia.
scoring_metric = 'accuracy' # Métrica para evaluar

# --- SFS ---
print(f"Iniciando Sequential Forward Selection (SFS) para {K_WRAPPER} features...")
print(f"Modelo: DecisionTree, Métrica: {scoring_metric}")
sfs = SequentialFeatureSelector(
    model_wrapper,
    n_features_to_select=K_WRAPPER,
    direction='forward',
    scoring=scoring_metric,
    cv=3, # 3-fold cross-validation
    n_jobs=-1 # Usar todos los cores
)

start_time = time.time()
sfs.fit(X_train_scaled, y_train)
end_time = time.time()

selected_indices_sfs = sfs.get_support(indices=True)
selected_features_sfs = X.columns[selected_indices_sfs]

print(f"SFS completado en {end_time - start_time:.2f} segundos.")
print(f"Características seleccionadas (SFS): {list(selected_features_sfs)}")

# --- SBE ---
print(f"\nIniciando Sequential Backward Elimination (SBE) para {K_WRAPPER} features...")
print(f"Modelo: DecisionTree, Métrica: {scoring_metric}")
sbe = SequentialFeatureSelector(
    model_wrapper,
    n_features_to_select=K_WRAPPER,
    direction='backward',
    scoring=scoring_metric,
    cv=3,
    n_jobs=-1
)

start_time = time.time()
sbe.fit(X_train_scaled, y_train)
end_time = time.time()

selected_indices_sbe = sbe.get_support(indices=True)
selected_features_sbe = X.columns[selected_indices_sbe]

print(f"SBE completado en {end_time - start_time:.2f} segundos.")
print(f"Características seleccionadas (SBE): {list(selected_features_sbe)}")



--- 4.1.1 Wrapper: Sequential Methods ---
Iniciando Sequential Forward Selection (SFS) para 5 features...
Modelo: DecisionTree, Métrica: accuracy


### Métodos Determinísticos (No implementados en `sklearn`)
* **Greedy hill climbing / Best first**: SFS y SBE son las implementaciones más comunes de estas estrategias.
* **Plus-L-Minus-r algorithm, Floating search selection (SFFS/SBFS)**: Son variaciones más avanzadas. La librería `mlxtend` (no disponible aquí) implementa SFFS y SBFS.
* **Tabu search, Branch and bound**: Son algoritmos de optimización global muy complejos, no estándar para feature selection en `sklearn`.


## 4.2 Heurísticas No Determinísticas (Metaheurísticas)

Esta categoría incluye todos los algoritmos estocásticos (basados en aleatoriedad) y poblacionales.
* **Single-Solution**: Simulated annealing, Las Vegas, GRASP, Variable neighborhood search.
* **Population-Based**: Scatter search, Ant colony optimization (ACO), Particle swarm optimization (PSO).
* **Evolutionary Algorithms**: Genetic algorithms (GA), Estimation of distribution algorithms (EDA), Differential evolution (DE), Genetic programming (GP), Evolution strategies (ES).

**Todos estos métodos requieren librerías especializadas** (ej. `DEAP`, `sklearn-genetic`, `pyswarms`, `mealpy`) o implementaciones manuales muy complejas. No están disponibles en `scikit-learn`.

A continuación, se muestra un *ejemplo conceptual* de cómo se usaría un Algoritmo Genético si la librería `genetic_selection` estuviera instalada.


In [ ]:
print("\n--- 4.2.1 Wrapper: Metaheurísticas (Ejemplo Conceptual) ---")
print("Las metaheurísticas (GA, PSO, ACO, etc.) NO están implementadas en scikit-learn.")
print("Requieren librerías de terceros como 'sklearn-genetic', 'DEAP', 'pyswarms', etc.")
print("A continuación se muestra un bloque de código comentado como ejemplo conceptual.")

# --- Código comentado (NO SE EJECUTA) ---
"""
try:
    # pip install sklearn-genetic
    from genetic_selection import GeneticSelectionCV
    
    print("Librería 'genetic_selection' encontrada. Ejecutando ejemplo de GA...")
    
    # Usar un estimador rápido
    estimator_ga = models["DecisionTree"]
    
    selector_ga = GeneticSelectionCV(
        estimator_ga,
        cv=3,
        scoring="accuracy",
        population_size=30, # Reducido para el ejemplo
        generations=20,     # Reducido para el ejemplo
        n_jobs=-1,
        verbose=0,
        caching=True
    )
    
    # selector_ga.fit(X_train_scaled, y_train)
    # selected_features_ga = X.columns[selector_ga.support_]
    # print(f"Características seleccionadas (GA): {list(selected_features_ga)}")
    print("Ejecución de GA omitida por tiempo, pero la librería está presente.")

except ImportError:
    print("Librería 'genetic_selection' no encontrada. Omitiendo ejemplo de GA.")
"""


# 5. Conclusión

Este notebook ha implementado los métodos de selección de características más comunes y accesibles de la lista proporcionada, utilizando `scikit-learn` y `scipy`.

**Resumen de Métodos Implementados:**
* **Filtro (Paramétrico):** ANOVA / t-test (`f_classif`), Chi-cuadrado (`chi2`), Información Mutua (`mutual_info_classif`).
* **Filtro (No Paramétrico):** Mann-Whitney y Kruskal-Wallis (de `scipy.stats`).
* **Wrapper (Determinístico):** Sequential Forward Selection (SFS) y Sequential Backward Elimination (SBE) (de `sklearn.feature_selection`).

Los métodos más avanzados (especialmente los wrappers no determinísticos y los filtros menos comunes) requerirían librerías especializadas o una implementación manual intensiva.
